# Extract frames from videos in a path
Set the input path that contains videos and the output path for frames


If cv2 is missing install opencv-python before running
Example pip install opencv-python


In [3]:
from pathlib import Path
import cv2

input_dir = Path(r"date/videos/")
output_dir = Path(r"date/frames")
target_fps = 1
recursive = False


In [4]:
video_exts = {".mp4", ".avi", ".mov", ".mkv", ".m4v", ".wmv"}

if not input_dir.exists():
    raise FileNotFoundError(f"input_dir not found {input_dir}")

pattern = "**/*" if recursive else "*"
videos = [p for p in input_dir.glob(pattern) if p.suffix.lower() in video_exts]

print(f"found {len(videos)} videos")
output_dir.mkdir(parents=True, exist_ok=True)

def extract_frames(video_path: Path, out_root: Path, fps_target: float) -> None:
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"could not open {video_path}")
        return
    name = video_path.stem
    out_dir = out_root / name
    out_dir.mkdir(parents=True, exist_ok=True)
    source_fps = cap.get(cv2.CAP_PROP_FPS)
    if not source_fps or source_fps <= 0:
        source_fps = 30.0
    step = max(1, int(round(source_fps / fps_target)))
    frame_idx = 0
    saved_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % step == 0:
            out_path = out_dir / f"{name}_{saved_idx:06d}.jpg"
            cv2.imwrite(str(out_path), frame)
            saved_idx += 1
        frame_idx += 1
    cap.release()
    print(f"{video_path.name} saved {saved_idx} frames")

fps_target = max(1e-6, float(target_fps))
for vp in videos:
    extract_frames(vp, output_dir, fps_target)


found 1 videos
Video_1_3(1).avi saved 143 frames
